# Analysis database Olist - Kaggle

In [14]:
#Packages
import geopy as geo
import pandas as pd
import numpy as np
import geopandas as gpd
from unidecode import unidecode 
import folium

#### Database source

In [2]:
customer = pd.read_csv('data/olist_customers_dataset.csv')
order = pd.read_csv('data/olist_orders_dataset.csv')
item = pd.read_csv('data/olist_order_items_dataset.csv')
products = pd.read_csv('data/olist_products_dataset.csv')
location= pd.read_csv('data/olist_geolocation_dataset.csv')
sellers = pd.read_csv('data/olist_sellers_dataset.csv')
product_info=pd.merge(products,item, on = 'product_id')
product_info = product_info[['product_id', 'product_category_name','order_id', 'order_item_id', 'seller_id','price', 'freight_value']]
product_info = pd.merge(product_info,order,on = 'order_id')
product_info = pd.merge(product_info,customer, on = 'customer_id')

#### Data Clean

In [3]:
# Applying unidecode
from unidecode import unidecode 
location['geolocation_city']= location['geolocation_city'].apply(lambda x : unidecode(x))
# Fixing some places
location.loc[430999,['geolocation_state']] = 'RJ'
location.loc[460406,['geolocation_state']] = 'RJ'
location.loc[72852,['geolocation_state']] = 'SP'
location.loc[840047,['geolocation_state']] = 'MS'
location.loc[22261,['geolocation_state']] = 'SP'
# Base location of places 
location = location.groupby(['geolocation_state','geolocation_city','geolocation_zip_code_prefix'],as_index=False).agg({"geolocation_lat" :np.mean,"geolocation_lng" :np.mean})
# Geo coordinate locationizations
gdf = gpd.GeoDataFrame(location, geometry=gpd.points_from_xy(location.geolocation_lng, location.geolocation_lat, crs = 'EPSG:4326'))
location['estado-cidade'] = location['geolocation_state'] + "-"+ location['geolocation_city']
#Capitals Location
capitais_location = location.loc[location['estado-cidade'].isin(['AC-rio branco','AL-maceio','AP-macapa','AM-manaus','BA-salvador','CE-fortaleza','ES-vitoria','GO-goiania','MA-sao luis','MT-cuiaba','MS-campo grande','MG-belo horizonte','PA-belem','PB-joao pessoa','PR-curitiba','PE-recife','PI-teresina','RJ-rio de janeiro','RN-natal','RS-porto alegre','RO-porto velho','RR-boa vista','SC-florianopolis','SP-sao paulo','SE-aracaju','TO-palmas','DF-brasilia'])][['geolocation_state','geolocation_city','geolocation_lat','geolocation_lng']].drop_duplicates()
capitais_location  = capitais_location.groupby(['geolocation_state','geolocation_city'], as_index=False).agg({'geolocation_lat':np.average,'geolocation_lng':np.average})

## Data Transformation and extraction of sales information

In [4]:
# Creation of daframe with what each sellers have
seller_info = pd.merge(sellers,product_info, on = 'seller_id')
# Grouping by the amount of order that the sellers had
seller_sales = seller_info.groupby(by = (['seller_id','seller_city','seller_state','product_category_name','seller_zip_code_prefix']), as_index= False).agg({'price': np.sum, 'freight_value': np.sum,'order_id' :np.size})
seller_sales = seller_sales.rename(columns ={'order_id': 'Amount of orders','price':'Orders cost(R$)', 'freight_value':'Freight cost(R$)'})
# Getting the average cost of the the orders 
seller_sales['Average cost of orders'] = round(seller_sales['Orders cost(R$)']/seller_sales['Amount of orders'],2)
# Getting the total value 
seller_sales['Total Cost(R$)'] = seller_sales['Orders cost(R$)'] + seller_sales['Freight cost(R$)']
# Getting sellers information 
seller_cnpj = seller_info[['seller_id','product_category_name','seller_zip_code_prefix','seller_city','seller_state']].drop_duplicates()
# Getting sellers category
seller_category = seller_info[['seller_id','product_category_name']].drop_duplicates()
# Amount of seller by each category
seller_qty_category= seller_category.groupby('seller_id',as_index = False).size()
# Seller sales numbers 
seller_sales.sort_values(["Amount of orders","Orders cost(R$)","Freight cost(R$)"], ascending=False,ignore_index=True).head(20)
seller_sales_st = seller_info.groupby(by = (['seller_state','product_category_name']), as_index= False).agg({'price': np.sum, 'freight_value': np.sum,'order_id' :np.size,'seller_id':np.size})

## Classes

#### 1 - Estimation of the distance of each customer to the sellers

In [5]:
class sellers_information():

    def __init__(self,seller):
        self.__seller = seller

    @property
    def seller_location(self):
            seller_location = sellers.loc[sellers['seller_id'].isin(self.__seller)]
            return seller_location
    
    def mycustomers(self):
        ''' 
            This function will calculate the distance to from the seller for each customer in the database. 
            The seller code is the mandatory field this function is using the Projected coordinate system for Brazil : EPSG 31983 (SIRGAS 2000). 
            The distance formula is : 
                Distance = (RadiusEarthKM * (2 * ArcSin(Sqr((Sin((Lat1 - Lat2)/2)2) + Cos(Lat1) * Cos(Lat2) * (Sin((Lon1 - Lon2)/2)2)))))
            The function will return a pandas dataframe with the sellers id, customer id, customer city, customer state, the average price of this product category amd the freight cost.
        '''        
        try:
            # Verification if the seller id is the base    
            seller_location = sellers.loc[sellers['seller_id'].isin(self.__seller)]
            # Getting the seller orders
            seller_orders = product_info.loc[product_info['seller_id'].isin(self.__seller) ]
            # Identifying each customer that bought any product that the seller has
            clients = seller_orders.loc[seller_orders['seller_id'].isin(self.__seller)]['customer_id']
            # Identification of each customer that are in the database
            clients_location = customer.loc[customer['customer_id'].isin(clients)]
            # Getting the seller location
            seller_location = pd.merge(seller_location,location, how = 'left', left_on ='seller_zip_code_prefix', right_on= 'geolocation_zip_code_prefix')
            # Tranforming the geo coordinates to geopandas dataframe with EPSG: 4326
            seller_location =  gpd.GeoDataFrame(seller_location, geometry=gpd.points_from_xy(seller_location.geolocation_lng, seller_location.geolocation_lat, crs = 'EPSG:4326'))
            # Transforming the geo points to EPSG 31983
            seller_location = seller_location.to_crs('EPSG:31983')
            # Getting customers locations 
            clients_location = pd.merge(clients_location,location, how = 'left', left_on ='customer_zip_code_prefix', right_on= 'geolocation_zip_code_prefix')
            # Tranforming the geo coordinates to geopandas dataframe with EPSG: 4326
            clients_location =  gpd.GeoDataFrame(clients_location, geometry=gpd.points_from_xy(clients_location.geolocation_lng, clients_location.geolocation_lat, crs = 'EPSG:4326'))
            # Transforming the geo points to EPSG 31983
            clients_location = clients_location.to_crs('EPSG:31983')
            # Duplicates remotion
            seller_location= seller_location.drop_duplicates(subset='seller_id')
            clients_location = clients_location.drop_duplicates(subset = 'customer_unique_id')
            # Creation of distance dataframe
            df_distance = pd.DataFrame(columns=['seller_id','customer_unique_id','distance (KM)'])
            # Identifying sellers and their clients 
            seller_ids = seller_location['seller_id'].tolist()
            client_ids = clients_location['customer_unique_id'].tolist()
            for s_id in seller_ids:
                seller_pt = seller_location[seller_location.seller_id==s_id]['geometry'].reset_index()
                # Distance calculation
                for c_id in client_ids:
                    client_pt = clients_location[clients_location.customer_unique_id==c_id]['geometry'].reset_index()
                    d = seller_pt.distance(client_pt).tolist()[0] 
                    df_length = len(df_distance)
                    df_distance.loc[df_length] = [s_id,c_id,d]
            # Dataframe Creation
            df_distance = pd.merge(df_distance,clients_location,on = 'customer_unique_id')
            df_distance = pd.merge(seller_orders,df_distance, on = (['seller_id','customer_unique_id']))
            df_distance =  df_distance[['seller_id','customer_unique_id', 'distance (KM)','customer_city_x','customer_state_x','price', 'freight_value']]
            df_distance['distance (KM)'] = (df_distance['distance (KM)']/1000).round(2)
            df_distance = df_distance.rename(columns ={'customer_city_x':'customer_city','customer_state_x':'customer_state','price': 'order_value'})
            return df_distance
        except:
            msg = 'Sales id not found'
            return msg

####  Sellers informations class example

In [7]:
x = sellers_information(sellers.head(2)['seller_id'])

x.mycustomers().head()

,seller_id,customer_unique_id,distance (KM),customer_city,customer_state,order_value,freight_value
0,d1b65fc7debc3361ea86b5f14c68d2e2,fbc9c3a06d228159f866fd7bcd32febe,57.38,campinas,SP,199.90,23.40
1,d1b65fc7debc3361ea86b5f14c68d2e2,be1401bbfd64455c798bb4683e915c61,391.87,rio de janeiro,RJ,89.99,45.09
2,d1b65fc7debc3361ea86b5f14c68d2e2,f95a2fb1caacbd10edbe4afa7a5980f8,80.17,piracicaba,SP,99.90,22.70
3,d1b65fc7debc3361ea86b5f14c68d2e2,fe99c37acdbd978fecb62eea4c58f415,814.04,boa esperanca,ES,89.99,45.09
4,d1b65fc7debc3361ea86b5f14c68d2e2,8715603183f796a544107275fdb734c2,124.91,sao paulo,SP,89.99,24.94


### 2. Class to choose the best option to buy or the closest seller or the best price

In [8]:
class best_buy():

    def __init__(self,client,category,no_register,type):
        self.__client = client
        self.__category = category
        self.__no_register = no_register
        self.__type = type
    
    def best_solution(self):
        ''' 
        This function enable to look for the best price for clients and for the closest seller or just the closest. In the case of best price, the priority will be the price following the minimal distance. 
        The arguments are mandatory : Client ID, product category, amount of sellers ,and type of buy fast or best_price.
        This function is using the Projected coordinate system for Brazil : EPSG 31983 (SIRGAS 2000). 
        The distance formula is : 
            Distance = (RadiusEarthKM * (2 * ArcSin(Sqr((Sin((Lat1 - Lat2)/2)2) + Cos(Lat1) * Cos(Lat2) * (Sin((Lon1 - Lon2)/2)2)))))
        The function will return a pandas dataframe with the customer id, customer city, customer state, sellers id, the amount of orders of this seller, distance, the average price of this product category amd the freight cost.
        '''
        # Customer identification 
        customer_location = customer.loc[customer['customer_unique_id']==self.__client]
        # Product category identification
        seller_cat = seller_cnpj.loc[seller_cnpj['product_category_name'] ==self.__category]
        # Sellers category identification
        seller_venda_in = seller_sales.loc[seller_sales['product_category_name'] ==self.__category]
        # Seller location 
        seller_location= pd.merge(seller_cat,location, how = 'left', left_on ='seller_zip_code_prefix', right_on= 'geolocation_zip_code_prefix')
        # Tranforming the geo coordinates to geopandas dataframe with EPSG: 4326
        seller_location=  gpd.GeoDataFrame(seller_location, geometry=gpd.points_from_xy(seller_location.geolocation_lng, seller_location.geolocation_lat, crs = 'EPSG:4326'))
        # Seller location transformation to EPSG 31983
        seller_location= seller_location.to_crs('EPSG:31983')
        # Client location 
        customer_location = pd.merge(customer_location,location, how = 'left', left_on ='customer_zip_code_prefix', right_on= 'geolocation_zip_code_prefix')
        # Client location transformation to EPSG 31983
        customer_location =  gpd.GeoDataFrame(customer_location, geometry=gpd.points_from_xy(customer_location.geolocation_lng, customer_location.geolocation_lat, crs = 'EPSG:4326'))
        customer_location = customer_location.to_crs('EPSG:31983')
        # Duplicates remotion
        seller_location= seller_location.drop_duplicates(subset='seller_id')
        customer_location = customer_location.drop_duplicates(subset = 'customer_unique_id')
        # Creation of distance dataframe
        df_distance = pd.DataFrame(columns=['customer_unique_id','seller_id','Distance (KM)'])
        # Sending the sellers and client 
        seller_ids = seller_location['seller_id'].tolist()
        client_ids = customer_location['customer_unique_id'].tolist()
        # Identifying each seller to the client
        for c_id in client_ids:
            client_pt = customer_location[customer_location.customer_unique_id==c_id]['geometry'].reset_index()
            # Distance calculation
            for s_id in seller_ids:
                    seller_pt = seller_location[seller_location.seller_id==s_id]['geometry'].reset_index()
                    d = client_pt.distance(seller_pt).tolist()[0]
                    df_length = len(df_distance)
                    df_distance.loc[df_length] = [c_id,s_id,d]
        # Dataframe creation
        df_distance = df_distance.drop_duplicates()
        df_distance = pd.merge(df_distance,customer_location,on = 'customer_unique_id')
        df_distance = pd.merge(seller_venda_in  ,df_distance, on = ('seller_id'))
        df_distance = df_distance[['customer_unique_id', 'customer_city','customer_state','seller_id','product_category_name', 'seller_city', 'seller_state','Amount of orders','Orders cost(R$)','Freight cost(R$)','Distance (KM)','Total Cost(R$)','Average cost of orders']]
        df_distance['Distance (KM)'] = (df_distance['Distance (KM)']/1000).round(2)
        # Type of buy 
        if self.__type == 'fast':
            df_distance = df_distance.sort_values(["Distance (KM)"], ascending=True,ignore_index=True).head(self.__no_register)
        elif self.__type == 'best_price':
            df_distance = df_distance.sort_values(["Average cost of orders","Distance (KM)"], ascending=True,ignore_index=True).head(self.__no_register)
        return df_distance

    def best_map(self):
        ''' 
        This function will return a geografical map with the customer location and the seller. 
        The customer id is the first mandatory argument, the product category is the second mandatory field and the distance dataframe from the responsive function is the third mandatory argument.
        The map will be created using the free OpenStreetMap API, the blue marker represents the customer and the green markers represents the sellers.
        '''
        # Values of reponsive 
        d = self.best_solution()
        # Identifying the customer 
        customer_location = customer.loc[customer['customer_unique_id']==self.__client]
        # Getting the customer zip code 
        customer_location = pd.merge(customer_location,location, how = 'left', left_on ='customer_zip_code_prefix', right_on= 'geolocation_zip_code_prefix')
        # Tranforming the geo coordinates to geopandas dataframe with EPSG: 4326
        customer_location =  gpd.GeoDataFrame(customer_location, geometry=gpd.points_from_xy(customer_location.geolocation_lng, customer_location.geolocation_lat, crs = 'EPSG:4326'))
        # Customer location transformation to EPSG 31983
        customer_location = customer_location.to_crs('EPSG:31983')
        # Identifying the sellers that fulfill the order
        seller_cat = seller_cnpj.loc[seller_cnpj['product_category_name'] ==self.__category]
        # Getting the seller zip code 
        seller_location= pd.merge(seller_cat,location, how = 'left', left_on ='seller_zip_code_prefix', right_on= 'geolocation_zip_code_prefix')
        # Tranforming the geo coordinates to geopandas dataframe with EPSG: 4326
        seller_location=  gpd.GeoDataFrame(seller_location, geometry=gpd.points_from_xy(seller_location.geolocation_lng, seller_location.geolocation_lat, crs = 'EPSG:4326'))
        # Customer location transformation to EPSG 31983
        seller_location= seller_location.to_crs('EPSG:31983')
        # Sending the values a list and using the API to display a map
        coordinates = seller_location.loc[seller_location['seller_id'].isin(d['seller_id'])][['geolocation_lat','geolocation_lng']].values.tolist()
        # Type of buy
        if self.__type == 'fast':
            p_zoom_start = 13
        elif self.__type == 'best_price':
            p_zoom_start = 7   
        # Creation of the map
        m = folium.Map(location=[customer_location['geolocation_lat'],customer_location['geolocation_lng']],
                   API_key=  "AjPvKPz1j2wCaJdCMYdWL3XaRpCMhbdyhArjG7VIMHflQuqcZY4HrMnn9vGhJWQC",zoom_start=p_zoom_start)
        # Addition of the client marker in the map
        folium.Marker([customer_location['geolocation_lat'],customer_location['geolocation_lng']], popup="<i> customer localization </i>").add_to(m)
        # Addition of the sellers in the map
        for i, j in coordinates:
            folium.Marker([i,j], popup="<i> seller localization </i>",icon=folium.Icon(color='green')).add_to(m)
        return m 

In [16]:
client = '861eff4711a542e4b93843c6dd7febb0'
category = 'beleza_saude'
no_register = 5
type = 'fast'

john = best_buy(client,category,no_register,type)

john.best_map() # To see the map visit the : https://nbviewer.org/github/perinpaulo1/perinpaulo1/blob/main/olist_data_case.ipynb


In [17]:
client = '861eff4711a542e4b93843c6dd7febb0'
category = 'beleza_saude'
no_register = 5
type = 'best_price'

john = best_buy(client,category,no_register,type)

john.best_map()  # To see the map visit the : https://nbviewer.org/github/perinpaulo1/perinpaulo1/blob/main/olist_data_case.ipynb